<a href="https://colab.research.google.com/github/andartya-setyawan/TubesAnalisaKeputusanTI/blob/main/Analisis_Pasar_Mobil_Australia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Besar: Analisis Pasar Mobil Australia

Nama: Andartya Setyawan Darna<br>
NIM: 202022420016
---
Proyek ini berfokus pada analisis data pasar mobil di Australia (Australia Car Market Data). Dataset ini berisi ribuan entri mengenai berbagai jenis kendaraan yang dijual, mencakup berbagai merek, model, dan spesifikasi teknis lainnya. Tujuan utama dari analisis ini adalah memanfaatkan teknik Machine Learning untuk memprediksi kategori harga mobil (apakah termasuk kategori High-End/Mahal atau Budget/Terjangkau) berdasarkan karakteristik kendaraan tersebut.

Dengan memanfaatkan dataset yang mencakup berbagai informasi penting seperti merek, tahun pembuatan, tipe transmisi, tipe bodi, jarak tempuh, dan spesifikasi mesin, kami bertujuan untuk mengidentifikasi pola dan wawasan yang dapat memberikan pemahaman mendalam mengenai valuasi kendaraan. Melalui data preprocessing yang komprehensif, analisis data eksploratif (Exploratory Data Analysis/EDA), dan penerapan model klasifikasi Random Forest, proyek ini berupaya tidak hanya untuk memprediksi kategori harga dengan akurasi tinggi, tetapi juga mengungkap faktor-faktor mendasar yang paling berpengaruh terhadap penentuan harga mobil di pasaran.

Dengan memanfaatkan kemampuan bahasa pemrograman Python, library pandas untuk manipulasi data, matplotlib dan seaborn untuk visualisasi, serta Scikit-Learn sebagai backend untuk pemodelan, kita akan menavigasi proses analisis data secara menyeluruh. Perjalanan ini akan membawa kita dari pemuatan data awal (data loading) dan pembersihan data (data cleaning) menuju pembangunan dan evaluasi model, dengan tujuan akhir memberikan wawasan prediktif yang berharga dalam domain pasar otomotif.

In [ ]:
# ==========================================
# 2. DATA LOADING & INITIAL EXPLORATION
# ==========================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Membaca Data
# Memastikan file 'Australian Vehicle Prices.csv' sudah diupload
filename = 'australia_car_market.csv'

try:
    df = pd.read_csv(filename)
    print("Data berhasil dimuat!")
except FileNotFoundError:
    print("File tidak ditemukan. Mohon upload file dataset terlebih dahulu.")

# Menampilkan 5 baris pertama data mentah
print("\n--- Tampilan Data Mentah (5 Teratas) ---")
display(df.head())

# Cek tipe data awal
print("\n--- Info Data Sebelum Cleaning ---")
print(df.info())

# ==========================================
# DATA CLEANING (PEMBERSIHAN DATA)
# ==========================================
# Masalah pada dataset ini:
# Kolom 'Price' dan 'Kilometers' masih dianggap teks (Object) karena ada simbol ($, koma, km).
# Kita harus mengubahnya menjadi angka (Float/Integer).

print("\n... Sedang melakukan pembersihan data ...")

# 1. Hapus baris yang nilai Price atau Kilometers-nya kosong (NaN)
df = df.dropna(subset=['Price', 'Kilometers'])

# 2. Bersihkan Kolom Price
# Hapus semua karakter yang bukan angka atau titik
df['Price'] = df['Price'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
# Ubah menjadi tipe data angka (float)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# 3. Bersihkan kolom Kilometers
# Hapus karakter 'km', koma, dll
df['Kilometers'] = df['Kilometers'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
df['Kilometers'] = pd.to_numeric(df['Kilometers'], errors='coerce')

# 4. Hapus data yang gagal diubah (menjadi NaN)
df = df.dropna(subset=['Price', 'Kilometers'])

# Tampilkan hasil setelah dibersihkan
print("Data berhasil dibersihkan!")
print(f"Ukuran data sekarang: {df.shape[0]} baris, {df.shape[1]} kolom")

print("\n--- Info Data Setelah Cleaning ---")
print(df.info())

Data berhasil dimuat!

--- Tampilan Data Mentah (5 Teratas) ---


,ID,Name,Price,Brand,Model,Variant,Series,Year,Kilometers,Type,Gearbox,Fuel,Status,CC,Color,Seating Capacity
0,11530130,2010 Toyota Rukus Build 2 AZE151R,9999,Toyota,Rukus,Build 2,AZE151R,2010,263000,Wagon,Automatic,Unleaded Petrol,Used,2362,Grey,5
1,12190570,2021 Mercedes-Benz V 250 D Avantgarde MWB 447 ...,117990,Mercedes-Benz,V,250 D Avantgarde MWB,447 MY21,2021,19,Wagon,Automatic,Diesel,New In Stock,2143,Black,7
2,12321855,2021 Mercedes-Benz Valente 116 CDI MWB RWD 447...,79990,Mercedes-Benz,Valente,116 CDI MWB RWD,447 MY21,2021,24,Wagon,Automatic,Diesel,New In Stock,2143,Black,8
3,12346971,2010 Mercedes-Benz E250 CDI Avantgarde 207,34990,Mercedes-Benz,E250,CDI Avantgarde,207,2010,120579,Cabriolet,Automatic,Diesel,Used,2143,Black,4
4,12363884,2016 Holden Cruze CD JH MY16,15990,Holden,Cruze,CD,JH MY16,2016,72506,Sportswagon,Automatic,Unleaded Petrol,Used,1796,White,5



--- Info Data Sebelum Cleaning ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17048 entries, 0 to 17047
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                17048 non-null  int64 
 1   Name              17048 non-null  object
 2   Price             17048 non-null  int64 
 3   Brand             17048 non-null  object
 4   Model             17048 non-null  object
 5   Variant           17048 non-null  object
 6   Series            17048 non-null  object
 7   Year              17048 non-null  int64 
 8   Kilometers        17048 non-null  int64 
 9   Type              17048 non-null  object
 10  Gearbox           17048 non-null  object
 11  Fuel              17048 non-null  object
 12  Status            17048 non-null  object
 13  CC                17048 non-null  int64 
 14  Color             17048 non-null  object
 15  Seating Capacity  17048 non-null  int64 
dtypes: int64(6), object(10

In [ ]:
# ==========================================
# 3. STATISTICAL SUMMARY OF THE DATASET
# ==========================================

# A. Statistik untuk Data Numerik (Angka)
# Baris ini akan menampilkan rata-rata (mean), standar deviasi, nilai min/max
print("--- Ringkasan Statistik Data Numerik ---")
display(df.describe())

# B. Statistik untuk Data Kategorikal (Teks)
# Baris ini akan menampilkan jumlah variasi (unique), data terbanyak (top), dan frekuensinya
print("\n--- Ringkasan Statistik Data Kategorikal ---")
# Ambil kolom tipe object saja
display(df.describe(include=['object']))

# C. Analisis Singkat (Opsional, untuk ditampilkan di output)
print("\n--- Analisis Singkat ---")
print(f"Rata-rata harga mobil: {df['Price'].mean():,.2f}")
print(f"Mobil tertua tahun: {df['Year'].min()}")
print(f"Mobil terbaru tahun: {df['Year'].max()}")

--- Ringkasan Statistik Data Numerik ---


,ID,Price,Year,Kilometers,CC,Seating Capacity
count,1.704800e+04,17048.000000,17048.000000,1.704800e+04,17048.000000,17048.000000
mean,1.279027e+07,36777.778038,2015.481288,1.032314e+05,2491.830303,5.115849
std,5.051111e+04,30305.015328,4.721591,8.041313e+04,881.985562,1.121791
min,1.153013e+07,1000.000000,1989.000000,1.000000e+00,875.000000,2.000000
25%,1.275715e+07,18800.000000,2013.000000,4.450225e+04,1987.000000,5.000000
50%,1.280207e+07,29990.000000,2016.000000,8.845400e+04,2354.000000,5.000000
75%,1.283131e+07,45990.000000,2019.000000,1.488735e+05,2981.000000,5.000000
max,1.285246e+07,999000.000000,2022.000000,2.700000e+06,7300.000000,14.000000



--- Ringkasan Statistik Data Kategorikal ---


,Name,Brand,Model,Variant,Series,Type,Gearbox,Fuel,Status,Color
count,17048,17048,17048,17048,17048,17048,17048,17048,17048,17048
unique,8575,59,622,2363,2286,56,5,7,3,235
top,2018 Nissan X-Trail ST (2WD) T32 Series 2,Toyota,Ranger,S,MY18,Wagon,Automatic,Unleaded Petrol,Used,White
freq,33,2768,648,287,172,7768,14578,7016,16304,6177



--- Analisis Singkat ---
Rata-rata harga mobil: 36,777.78
Mobil tertua tahun: 1989
Mobil terbaru tahun: 2022


### Penjelasan Ringkasan Statistik

Dari hasil output statistik di atas, kita dapat menyimpulkan beberapa hal mengenai dataset ini:

1.  **Data Numerik:**
    * **Count:** Menunjukkan jumlah total data mobil yang siap dianalisis.
    * **Mean (Rata-rata):** Memberikan gambaran rata-rata harga pasaran mobil dan rata-rata jarak tempuh (*kilometers*) mobil bekas di Australia.
    * **Min & Max:** Kita bisa melihat rentang tahun pembuatan mobil (dari tahun berapa hingga tahun berapa) serta rentang harga termurah hingga termahal.

2.  **Data Kategorikal:**
    * **Unique:** Menunjukkan seberapa banyak variasi merek (*Brand*) atau tipe transmisi yang ada.
    * **Top:** Menunjukkan merek atau tipe mobil apa yang paling mendominasi pasar (paling sering muncul dalam dataset).
    * **Freq:** Frekuensi kemunculan dari data terbanyak tersebut.